### Resources:

* http://www.twilightlandscapes.com/IRAFtutorial/IRAFintro_06.html

In [ ]:
import os
import glob 

# Delete some directories/files from previous runs.

os.system("rm -rf login.cl database pyraf uparm")
os.system("mkiraf")


import astropy.io.fits as fits
%pylab
%matplotlib inline

# Dark subtract and flat field the spectra
from os.path import expanduser
home = expanduser("~/")
data_dir=home+"/data_home/ATC2017/61inch/23june_Spectra/"
pixelsize = 0.020#mm/pix
dispersion9_34 = 120.22*pixelsize#(A/mm)*mm/pix
dispersion11_23 = 121.08*pixelsize
center11_23 = 6000 #angstrom
center9_34 = 5000


In [ ]:
ls ~/data_home/ATC2017/61inch/23june_Spectra/

In [ ]:
# The name of the science file.
dataset="23june_wideslit_11grating-"
target="Vega"
filename = dataset+target+ '.fits'
extracted_filename =  dataset+target+'.ms.fits'
calibrated_filename =  dataset+target+'.calib.fits'
dispersion=dispersion11_23
crval=center11_23
text_file = open("filename.txt", "w")
text_file.write(filename)
text_file.close()

import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects,

with open('filenames.pickle', 'w') as f:  # Python 3: open(..., 'wb')
    pickle.dump([dataset, target, filename,extracted_filename,
                 calibrated_filename,
                 crval,
                 dispersion], f)



In [ ]:
datafiles=glob.glob(data_dir+dataset+"*"+target+'.fit')
median_data=np.median(np.array([fits.getdata(dfile) for dfile in datafiles]  ),axis=0)

In [ ]:
darkfiles=glob.glob(data_dir+"23june_wideslit_11grating-*dark1.fit")
median_dark=np.median(np.array([fits.getdata(dfile) for dfile in darkfiles]  ),axis=0)

## Use astropy to do initial data reduction

In [ ]:
plt.figure(figsize=[8,3])
plt.subplot(121)
plt.imshow(np.log10(median_data))
plt.colorbar()


plt.subplot(122)
dark_sub_data = median_data - median_dark
#float_ is necessary to change the byte order, e.g.
#https://github.com/mperrin/poppy/issues/213#issuecomment-289166554
plt.imshow(np.log10(dark_sub_data))#,vmax=6e5,vmin=0 ) 
plt.colorbar()

fits.writeto(filename,data=dark_sub_data,clobber=True)

## Use pyraf to analyze the spectrum

In [ ]:
%%bash
xterm -e python identify.py

In [ ]:
# Now load IRAF
import pyraf.iraf as iraf

# Load the packages we might need.

iraf.noao(_doprint=0)
iraf.onedspec(_doprint=0)
iraf.twodspec(_doprint=0)
iraf.apextract(_doprint=0)
iraf.unlearn(iraf.apall)
iraf.unlearn(iraf.autoidentif)


In [ ]:
# Delete previous results.

os.system("rm "+extracted_filename+" "+calibrated_filename)

# Make sure that the dispersion axis is in the header.

iraf.hedit(images=[filename], fields=["DISPAXIS"], value=["1"], add="Yes")

Commands for graphics windows:
1. `w` to enter zoom mode 
2. `e` in the bottom left and top right corner of the box you want zoom in to.
3. `w a` reset the zoom
4. `m` centroid and define center of the analysis aperture
5. `b` advance to setting the background
6. `s` select background
7. `f` fit background
8. `q` to advance to the next stage
9. `:or` increase order of polynomial fit


In [ ]:
# Run the spectral extraction program.

iraf.apextract.setParam("dispaxis", "1")

iraf.apall(input=filename, find="No", recenter="No", resize="No",interactive="Yes")
filename

In [ ]:
pixelsize = 0.020#mm/pix
dispersion9_34 = 120.22*pixelsize#(A/mm)*mm/pix
dispersion11_23 = 121.08*pixelsize
center11_23 = 6000 #angstrom
center9_34 = 5000
dispersion11_23

In [ ]:

iraf.identify(filename[:-5], coordli=home+"/Downloads/HgNe(1).dat",
              section="line 105 125",)

In [ ]:
iraf.autoidentify(filename[:-5], coordli=home+"/Downloads/HgNe(1).dat",
              section="line 265 285",
                  interactive='yes',
                  crval=center11_23,
                  cdelt=dispersion11_23,
                  fwidth=5)


In [ ]:
ls

In [ ]:
# Tell the extracted spectrum what the wavelength solutions are.

iraf.hedit(images=[extracted_filename],
           fields=["REFSPEC1"], \
        value=[filename], add="Yes")

iraf.dispcor(input=extracted_filename,
             output=calibrated_filename)

# Plot the extracted spectrum?

In [ ]:
iraf.splot(calibrated_filename)

In [ ]:
#when in doubt:
iraf.flpr()
iraf.flpr()
iraf.flpr()


In [ ]:
fits.getdata(extracted_filename)

In [ ]:
plt.plot(fits.getdata(extracted_filename))
plt.ylim([0,7e5])


In [ ]:
plt.plot(fits.getdata(calibrated_filename))
plt.ylim([0,7e5])

In [ ]:
A0V=fits.getdata(home+"/Downloads/pickles_uk_9.fits")
plot(A0V["WAVELENGTH"],A0V["FLUX"])

In [ ]:
from scipy.interpolate import interp1d
A0V_interpolator = interp1d(A0V["WAVELENGTH"],A0V["FLUX"])


In [ ]:
cal=fits.open(extracted_filename)
calibrated_filename

In [ ]:
plt.plot(cal[0].data)

In [ ]:
cal[0].header